## Test optimisation algorithms against results produced by model with known parameters
Using a similar approach to the preceding notebooks ($i_t = R_t\sum_{\tau<t} i_\tau g_{t-\tau}$),
test various optimisation algorithms using model-created outputs
(so that we know the ideal parameters that the algorithm should be approaching).
So that we can start using these various model components in more sophisticated ways,
I'll begin writing the code as properly encapsulated functions
and tracking a few key model quantities.

In [ ]:
from typing import List
import numpy as np
import pandas as pd
pd.options.plotting.backend = 'plotly'
from scipy.optimize import minimize, shgo
import nevergrad as ng

from emu_renewal.renew import RenewalModel
from emu_renewal.outputs import Outputs, plot_output_fit

In [ ]:
# def model_func(gen_time_mean: float, gen_time_sd: float, process_req: List[float], pop: int, seed: int, n_times: int) -> tuple:
#     """The common features of the model.
#     Get the generation time distribution from the user request,
#     linearly interpolate the non-mechanistic process,
#     exponentiate the result and run the renewal process.
#     """
#     gen_time_densities = get_gamma_densities_from_params(gen_time_mean, gen_time_sd, n_times)
#     req_x_vals = np.linspace(0.0, n_times, len(process_req))
#     func = get_linear_interp_func(req_x_vals, process_req)
#     process_vals = func(np.array([float(t) for t in range(n_times)]))
#     process_vals_exp = np.exp(np.array(process_vals))
#     model_result = renew_basic(gen_time_densities, process_vals_exp, pop, seed, n_times)
#     return model_result, process_vals_exp

# def process_calib_func(parameters: List[float], gen_time_mean, gen_time_sd, pop: int, seed: int, n_times: int, targets: dict) -> float:
#     """Only optimise the non-mechanistic process parameters,
#     least squares loss function.
#     """
#     incidence = model_func(gen_time_mean, gen_time_sd, parameters, pop, seed, n_times)[0][0]
#     return sum([(incidence[t] - d) ** 2 for t, d in targets.items()])    

# def all_param_calib_func(parameters: List[float], pop: int, seed: int, n_times: int, targets: dict) -> float:
#     """Additionally include the generation time parameters as parameters,
#     least squares loss function.
#     """
#     gen_time_mean, gen_time_sd, *process_req = parameters
#     incidence = model_func(gen_time_mean, gen_time_sd, process_req, pop, seed, n_times)[0][0]
#     return sum([(incidence[t] - d) ** 2 for t, d in targets.items()])

In [ ]:
test_process = [1.8, 2.5, 1.6, 0.7]
n_times = 40
model = RenewalModel(100.0, n_times, 10, 5)
gen_mean = 5.5
gen_sd = 1.8
seed = 1.0
test_results = model.func(gen_mean, gen_sd, np.log(test_process), seed)

def calib_func(parameters: List[float], targets: dict) -> float:
    incidence = model.func(gen_mean, gen_sd, parameters, seed).incidence
    return sum([(incidence[t] - d) ** 2 for t, d in enumerate(targets)])

### Local optimisation algorithm
#### Process values optimised only
Local optimisation algorithm.
It works for this simple case, but quickly breaks down in many other use cases.

In [ ]:
param_bounds = [[-1000.0, np.log(10.0)]] * 4
result = minimize(calib_func, [np.log(2.0)] * 4, method='Nelder-Mead', args=(test_results.incidence), bounds=param_bounds)
print(f'target: {test_process}')
print(f'result: {np.exp(result.x)}')

In [ ]:
plot_output_fit(pd.Series(test_results.incidence), test_results, 40, 1.0)

In [ ]:
param_bounds = [[-1000.0, np.log(10.0)]] * 4
result = minimize(process_calib_func, [np.log(2.0)] * 4, method='Nelder-Mead', args=(test_gen_mean, test_gen_sd, population, infectious_seed, n_times, test_vals), bounds=param_bounds)
model_result, process_vals = model_func(test_gen_mean, test_gen_sd, result.x, population, infectious_seed, n_times)
optimised, suscept, r_t = model_result
# plot_output_fit(test_vals, model_result, process_vals, n_times)

#### Generation time parameters included
The same local optimisation algorithm, 
but now additionally including generation time parameters in local optimisation.

In [ ]:
param_bounds = [[1.0, 10.0]] + [[1.0, 5.0]] + [[-10000.0, np.log(10.0)]] * 4
result = minimize(all_param_calib_func, [5.0, 1.5] + [np.log(2.0)] * 4, method='Nelder-Mead', args=(population, infectious_seed, n_times, test_vals), bounds=param_bounds)
model_result, process_vals = model_func(result.x[0], result.x[1], result.x[2:], population, infectious_seed, n_times)
optimised, suscept, r_t = model_result
# plot_output_fit(test_vals, model_result, process_vals, n_times)

### Global algorithm
Global optimisation with `scipy`'s `shgo` - need to capture arguments through closure due to bug in optimisation function
as per [comment found online](https://stackoverflow.com/questions/72794609/scipy-issue-passing-arguments-to-optimize-shgo-function).

In [ ]:
param_bounds = [[0.1, 10.0]] + [[0.1, 4.0]] + [[np.log(1.0), np.log(5.0)]] * 4
global_result = shgo(lambda x, p=population, s=infectious_seed, t=n_times, d=test_vals: all_param_calib_func(x, p, s, t, d), param_bounds)
model_result, process_vals = model_func(global_result.x[0], global_result.x[1], global_result.x[2:], population, infectious_seed, n_times)
optimised, suscept, r_t = model_result
print(test_process)
print(np.exp(np.array(global_result.x[2:])))
plot_output_fit(test_vals, model_result, process_vals, n_times)

### `nevergrad` algorithms
#### `NGOpt` with default settings
`NGOpt` applied without additional user requests.

In [ ]:
def obj_func(parameters):
    return all_param_calib_func(parameters, pop=1e2, seed=1.0, n_times=n_times, targets=test_vals)
optimizer = ng.optimizers.NGOpt(parametrization=6, budget=2000)
ngopt_result = optimizer.minimize(obj_func)
ngopt_output, ngopt_process = model_func(ngopt_result.value[0], ngopt_result.value[1], ngopt_result.value[2:], 1e2, 1.0, len(test_vals))
print(test_process)
print(np.exp(ngopt_result.value[2:]))
plot_output_fit(test_vals, ngopt_output, ngopt_process, n_times)

#### `NGOpt` with parameterisation
Specify starting points and bounds for algorithm.

In [ ]:
gen_time_mean_param = ng.p.Scalar(init=5.0, lower=0.1, upper=10.0)
gen_time_sd_param = ng.p.Scalar(init=1.0, lower=0.1, upper=4.0)
process_param = ng.p.Array(init=[0.0] * 4, lower=-10.0, upper=10.0)
instrum = ng.p.Instrumentation(gen_time_mean_param, gen_time_sd_param, process_param)
def obj_func(gen_time_mean, gen_time_sd, parameters):
    return all_param_calib_func([gen_time_mean, gen_time_sd] + list(parameters), pop=1e2, seed=1.0, n_times=n_times, targets=test_vals)
optimizer = ng.optimizers.NGOpt(parametrization=instrum, budget=2000)
ngopt_result = optimizer.minimize(obj_func).value[0]  # Zero index gets us the args (not kwargs)
ngopt_output, ngopt_process = model_func(ngopt_result[0], ngopt_result[1], ngopt_result[2], 1e2, 1.0, len(test_vals))
print(ngopt_result[0])
print(ngopt_result[1])
print(test_process)
print(np.exp(ngopt_result[2]))
plot_output_fit(test_vals, ngopt_output, ngopt_process, n_times)

#### `TwoPointsDE`
Doesn't work quite as well with `TwoPointsDE`.

In [ ]:
gen_time_mean_param = ng.p.Scalar(init=5.0, lower=0.1, upper=10.0)
gen_time_sd_param = ng.p.Scalar(init=1.0, lower=0.1, upper=4.0)
process_param = ng.p.Array(init=[0.0] * 4, lower=-10.0, upper=10.0)
instrum = ng.p.Instrumentation(gen_time_mean_param, gen_time_sd_param, process_param)
def obj_func(gen_time_mean, gen_time_sd, parameters):
    return all_param_calib_func([gen_time_mean, gen_time_sd] + list(parameters), pop=1e2, seed=1.0, n_times=n_times, targets=test_vals)
optimizer = ng.optimizers.TwoPointsDE(parametrization=instrum, budget=2000)
ngopt_result = optimizer.minimize(obj_func).value[0]
ngopt_output, ngopt_process = model_func(ngopt_result[0], ngopt_result[1], ngopt_result[2], 1e2, 1.0, len(test_vals))
print(ngopt_result[0])
print(ngopt_result[1])
print(test_process)
print(np.exp(ngopt_result[2]))
plot_output_fit(test_vals, ngopt_output, ngopt_process, n_times)

#### `NGOpt` optimisation with jittered synthetic data
Same as previous best algorithm, but with jitter applied to data.

In [ ]:
spread = 0.2
jitter = pd.Series(np.random.normal(scale=spread, size=n_times) + 1.0)
jittered_vals = test_vals * jitter
gen_time_mean_param = ng.p.Scalar(init=5.0, lower=0.1, upper=10.0)
gen_time_sd_param = ng.p.Scalar(init=1.0, lower=0.1, upper=4.0)
process_param = ng.p.Array(init=[0.0] * 4, lower=-10.0, upper=10.0)
instrum = ng.p.Instrumentation(gen_time_mean_param, gen_time_sd_param, process_param)
def obj_func(gen_time_mean, gen_time_sd, parameters):
    return all_param_calib_func([gen_time_mean, gen_time_sd] + list(parameters), pop=1e2, seed=1.0, n_times=n_times, targets=test_vals)
optimizer = ng.optimizers.NGOpt(parametrization=instrum, budget=2000)
jitter_result = optimizer.minimize(obj_func).value[0]
jitter_output, process_vals = model_func(jitter_result[0], jitter_result[1], jitter_result[2], 1e2, 1.0, len(test_vals))
print(test_process)
print(np.exp(jitter_result[2]))
plot_output_fit(jittered_vals, jitter_output, process_vals, n_times)